# Adapters: Bridging External Libraries

Molpy doesn't live in isolation. The `adapter` module provides **bidirectional bridges** to external molecular libraries, letting you leverage existing tools while staying in the MolPy ecosystem.

**Primary focus: RDKit integration**

RDKit is the industry-standard cheminformatics toolkit with powerful features for:
- SMILES parsing and  molecule building
- 2D/3D coordinate generation
- Substructure matching and fingerprinting
- Many specialized chemical algorithms

MolPy's RDKit adapter lets you tap into these capabilities while working primarily with `Atomistic` structures.

---

## Key Features

- **Bidirectional conversion** between RDKit `Mol` objects and MolPy `Atomistic` structures
- **3D coordinate generation** using RDKit's ETKDG conformer algorithm
- **Geometry optimization** with MMFF/UFF force fields
- **2D molecular drawings** (SVG export for visualization)
- **Stable atom mapping** for reliable round-trip conversion

**When to use adapters:**
- Starting from SMILES and need 3D coordinates
- Leveraging RDKit's chemistry algorithms (fingerprints, descriptors, etc.)
- Visualizing molecules with publication-quality 2D drawings
- Interfacing with external RDKit-based workflows

---

## Quick Start: SMILES to 3D Structure


In [ ]:
from rdkit import Chem
from molpy.adapter import RDKitWrapper, atomistic_to_mol, mol_to_atomistic

# 1. Create a molecule from SMILES (ethanol)
mol = Chem.MolFromSmiles("CCO")

# 2. Convert to MolPy Atomistic (2D structure, no coordinates yet)
atomistic = mol_to_atomistic(mol)
print(f"Converted molecule: {len(list(atomistic.atoms))} atoms")

# 3. Create wrapper for advanced operations
wrapper = RDKitWrapper.from_mol(mol)

# 4. Generate 3D coordinates and optimize geometry
wrapper.generate_3d(optimize=True)

# 5. Access the 3D MolPy structure
atomistic_3d = wrapper.inner
print(f"Generated 3D structure with coordinates")
print(f"Atoms: {len(list(atomistic_3d.atoms))}")
print(f"Bonds: {len(list(atomistic_3d.bonds))}")


### How 3D Generation Works

`wrapper.generate_3d(optimize=True)` performs these steps:

1. **Conformer sampling** – Generates initial 3D geometry using ETKDG algorithm
2. **Force field optimization** – Refines geometry using MMFF or UFF
3. **Coordinate assignment** – Updates the `Atomistic` structure with 3D coordinates

**Parameter choices:**
- `optimize=True` – Recommended for physically reasonable geometries
- `optimize=False` – Faster, but coordinates may have high energy

---

## Converting Back to RDKit

Once you've manipulated a structure in MolPy (e.g., polymer building, reactions), you can convert back to RDKit for further analysis:


In [ ]:
# Convert Atomistic back to RDKit Mol
mol_from_atomistic = atomistic_to_mol(atomistic_3d)
print(f"RDKit molecule: {mol_from_atomistic.GetNumAtoms()} atoms")

# Generate 2D drawing (SVG format)
svg = wrapper.draw(show=False)
print("Generated 2D molecular drawing (SVG)")

# Save to file if needed
# with open("molecule.svg", "w") as f:
#     f.write(svg)


## Typical Workflows

### Workflow 1: SMILES → MolPy → Simulation

```python
# 1. Start from SMILES
mol = Chem.MolFromSmiles("CCO")
wrapper = RDKitWrapper.from_mol(mol)

# 2. Generate 3D coordinates
wrapper.generate_3d(optimize=True)
atomistic = wrapper.inner

# 3. Convert to Frame for packing/export
# (see user-guide/builder.ipynb for packing)
# (see user-guide/io.ipynb for export to LAMMPS)
```

### Workflow 2: MolPy Structure → RDKit Analysis

```python
# 1. Build structure in MolPy (e.g., using reacter)
# atomistic = ... (from builder or reacter)

# 2. Convert to RDKit
mol = atomistic_to_mol(atomistic)

# 3. Use RDKit features
from rdkit.Chem import Descriptors
mw = Descriptors.MolWt(mol)
logp = Descriptors.MolLogP(mol)
```

---

## Notes on Atom Mapping

The adapter maintains **stable atom indices** during conversion:
- Atom `i` in `Atomistic` maps to atom `i` in RDKit `Mol`
- Bond information is preserved correctly
- This enables reliable round-trip conversion

**Limitations:**
- Some RDKit-specific properties (e.g., aromaticity flags) may not survive round-trip
- Coordinate precision is limited by file format if you export/re-import

---

## Future Adapters

The adapter pattern is extensible. Future versions may include:
- OpenBabel integration
- ASE (Atomic Simulation Environment) bridge
- MDAnalysis interoperability

If you need a specific adapter, check the [MolPy discussions](https://github.com/MolCrafts/molpy/discussions) or open a feature request!
